<a href="https://colab.research.google.com/github/leotuni/Nicolas-stuff/blob/master/SCTC_leo_21_06_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Error that needs solving : Data integrity

When importing with Yfinance or WebDatareader, we are getting slightly different datasets. (you need to reset runtime after running yfinance as the pdr ovveride fucks with stuff) 

In [ ]:
#@title for webDatreader
import pandas_datareader.data as web
import datetime as dt
start = dt.datetime.strptime('2018-3-22', '%Y-%m-%d')
stop = dt.datetime.strptime('2018-3-30', '%Y-%m-%d')
webdatadf= web.DataReader("BTC-EUR", "yahoo", start, stop) 
print (webdatadf)



In [ ]:
#@title for yahoo finance
!pip install -q yfinance
import yfinance as fyf 
from pandas_datareader import data as pdr 

fyf.pdr_override()
print (pdr.get_data_yahoo("BTC-EUR", start, stop))




#Error that needs solving: Integer underflow
#update: solved with new code (shift()) not sure if shift will create problems with the Nan now present at the end of the list.

```python
for i in range(len(data)):
        #if they intersect:
        if data.iloc[i, 1] > data.iloc[i, 2] and data.iloc[i-1, 1] < data.iloc[i-1,2]: 
            #Buy
            buy_price = data.iloc[i, 0]
            
            
        elif data.iloc[i, 1] < data.iloc[i, 2] and data.iloc[i-1, 1] > data.iloc[i-1,2]:

```



**data.iloc[i-1, 1]** results in -1 at position 0, meaning we are using the difference with the last element in the list. 

#Begin 


In [1]:
#@title imports 

import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
from scipy.stats import norm
import numpy as np 
import plotly.express as px


from itertools import product #ill move this to the top after



/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


## To read nicolas (after we cancel this section it) : 

The section "datframe setup", is a rewritten version of the section below of your code. I have saved the datfraems in a dictioanry, but we can use a list if you prefer. I dont think there is any need to save data, or download it, as it loads pretty fast. 

```python
# Soemer Torchia Crypto Trading (STCT)
# Create a object to be able to easily download the latest data
class Crypto_Asset:

    # Initializer / Instance Attributes
    def __init__(self, name):
        self.name = name
        
    # obtain Data
    def get_data(self):
        path = "/content/sample_data"
        file_path = path + "/cryptoDF-" + self.name + ".p"
        fyf.pdr_override()
        t_0 = datetime.date.today()
        #y_10 = t_0 - datetime.timedelta(days = 3652) # starting  10 years ago # see comment below
        y_10 = datetime.date(2018,2,1) # starting after the peak in Jan 2018
        # Check if pickle of past data already exists, in case it does only download missing data
        if os.path.isfile(file_path) == True: 
            cryptoDF = pickle.load( open( file_path, "rb" ) )
            #check if data is still up to date
            if cryptoDF.last_valid_index() != t_0:
                # if not, start downloading one day after the latest day in the cryptoDF, avoid doubling
                new_start_t = cryptoDF.last_valid_index() + datetime.timedelta(days = 1) 
                cryptoDF1 = pdr.get_data_yahoo( self.name, start= new_start_t) 
                cryptoDF.append(cryptoDF1) # append to bottom
        else:
            cryptoDF = pdr.get_data_yahoo( self.name, start= y_10)
            #cryptoDF['Returns'] = np.log(cryptoDF['Close'] / cryptoDF['Close'].shift(1)) # we might need this line for a different strategy
            #cryptoDF.dropna(inplace = True)
            pickle.dump( cryptoDF, open( file_path, "wb" ) )

        return cryptoDF  

symbols = ['BTC-EUR','ETH-EUR', 'LTC-EUR' , 'XRP-EUR', 'BCH-EUR','MIOTA-EUR' ]
btc = Crypto_Asset(symbols[0])
etc = Crypto_Asset(symbols[1])
ltc = Crypto_Asset(symbols[2])
xpr = Crypto_Asset(symbols[3])
bch = Crypto_Asset(symbols[4])
```

#Nicola's notes
##How far back should the data go? 

On the one hand you could argue the more the better,
on the other one might say that after the peak in the beginning of 2018 market behaviour has changed. 
Then some might say since the recent crash things are different ( i don t know if it is worth checking moments of the return time series, or average volumnes etc...

In [4]:
#@title Dataframe setup  { form-width: "30px" }
#the date in a nice date format
start_date_data = '2018-03-22' #@param {type:"date"} 
start = dt.datetime.strptime(start_date_data, '%Y-%m-%d') 
stop = dt.datetime.now()

#the symbols you gave me 
symbols = ['BTC-EUR','ETH-EUR', 'LTC-EUR' , 'XRP-EUR', 'BCH-EUR','MIOTA-EUR' ]
style.use('ggplot') #this is a cool style

#create a dictionary with all the stocks Q: do you prefer to work with lists of dictionaries ? 
dict_of_stock = { s: web.DataReader(s, "yahoo", start, stop) for s in (symbols) }
#or in list format:
# list_of_stock = [ web.DataReader(s, "yahoo", start, stop) for s in (symbols) ]


# Q: there are no missing values, so what is the drop nan for ? - ok i think you are dopping because of the SMA right ? 
# [df.isnull().sum() for df in dict_of_stock.values()] #evaluates no null values

# Simple Moving Average (SMA)  



1.   Calculates the values for the shorter SMA
2.   Calculates the values for the longer SMA.
3.   The calculation of the SMA values for two different rolling window sizes
4.   The derivation of the positions

The trading rules are:

*   Go long (= +1) when the shorter SMA is above the longer SMA.
*   Go short (= -1) when the shorter SMA is below the longer SMA.Optimization


A natural question that arises is if the chosen parameters SMA1=42 and SMA2=252 are the “right” ones. In general, investors prefer higher returns to lower returns ceteris paribus. Therefore, one might be inclined to search for those parameters that maximize the return over the relevant period. To this end, a brute force approach can be used that simply repeats the whole vectorized backtesting procedure for different parameter combinations, records the results, and does a ranking afterward. This is what the following code does:

1.   Specifies the parameter values for SMA1.
2.   Specifies the parameter values for SMA2.
3.   Combines all values for SMA1 with those for SMA2.
4.   Records the vectorized backtesting results in a DataFrame object.

The following code gives an overview of the results and shows the seven best-performing parameter combinations of all those backtested. The ranking is implemented according to the outperformance of the algorithmic trading strategy compared to the benchmark investment. The performance of the benchmark investment varies since the choice of the SMA2 parameter influences the length of the time interval and data set on which the vectorized backtest is implemented:

According to the brute force–based optimization, SMA1=40 and SMA2=190 are the optimal parameters, leading to an outperformance of some 230 percentage points. However, this result is heavily dependent on the data set used and is prone to overfitting.

A more rigorous approach would be to implement the optimization on one data set, the in-sample or training data set, and test it on another one, the out-of-sample or testing data set.

I would follow his advice but use the intersection of the SMAs for buying and selling

#For NIcolas, leo's notes Buy high Sell Low

I have rewritten the sectioin buy high sell low. Let me know if it makes sense. 

I have added a min_period to the rolling mean: it allows to not throw away data and have messy datasets, but since the period is smaller we have some overlap betwene datasets. We could also backfill. 


I have separated the conditions into a list. I then create a column['price']: 
*   it is positive when we are buying, 
*   0 when we do nothing, 
*   and negative when we are selling.

This stores all the data (i think? **can you confirm**). 

This linke of code: 
*   data['price'][:data.index.get_loc(data['price'].gt(0).idxmax())]

Makes sure that in case the first value is negative, it is replaced with a 0 

In [13]:
#@title  Buy Low and Sell High { form-width: "30px" }

sma1 = range(2, 60)  # maybe in steps of two days also fine
sma2 = range(60,180)

data=dict_of_stock['BTC-EUR']

for SMA1, SMA2 in product(sma1, sma2):
  #do the SMA 
  data['SMA1'] = data['Close'].rolling(SMA1,min_periods=1).mean() #made it with min periods cause it maintans the data better
  data['SMA2'] = data['Close'].rolling(SMA2,min_periods=1).mean()
  #these are the conditioins for to buy/sell
  conditions = [
  ((data['SMA1'] > data['SMA2']) & (data["SMA1"].shift(-1) < data["SMA2"].shift(-1))),
  ((data['SMA1'] < data['SMA2']) & (data["SMA1"].shift(-1) > data["SMA2"].shift(-1))),
  ]
  #The choices of what  you get back 
  choices = [data['Close'], -data['Close']]
  data['price'] = np.select(conditions, choices, default=0) #default value we do nothing
  
  #make sure that the first value is + (buy) (doesn't work yet cause of the multiple dates issues, but else this is the code: can just solve it by dropping non unique values, but not sure if they are important.)
  # data['price'][:data.index.get_loc(data['price'].gt(0).idxmax())] #gt means greater than 




data.head()


,High,Low,Open,Close,Volume,Adj Close,SMA1,SMA2,price
Date,,,,,,,,,
2019-01-11,3239.570801,3169.443604,3192.723145,3215.769775,4.830339e+09,3215.769775,3215.769775,3215.769775,0.0
2019-01-12,3225.897461,3186.506836,3215.427734,3192.196777,4.165968e+09,3192.196777,3203.983276,3203.983276,0.0
2019-01-13,3203.931396,3093.601318,3190.075684,3100.221680,4.084792e+09,3100.221680,3169.396077,3169.396077,0.0
2019-01-14,3249.573486,3098.569580,3104.109619,3230.380371,4.926029e+09,3230.380371,3184.642151,3184.642151,0.0
2019-01-15,3241.083252,3171.643799,3228.391113,3180.917969,4.851262e+09,3180.917969,3183.897314,3183.897314,0.0


# Continuation

At this point need to evaluate EARNINGS, Return: 
Earning = sum of all values
Returns  = Earning / sum positive values Prices 